# DatetimeOutputParser

The `DatetimeOutputParser` is an output parser that generates structured outputs in the form of `datetime` objects.

By converting the outputs of LLMs into `datetime` objects, it enables more systematic and consistent processing of date and time data, making it useful for data processing and analysis.

This tutorial demonstrates how to use the `DatetimeOutputParser` to:
1. Set up and initialize the parser for `datetime` generation
2. Convert a `datetime` object to a string

In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Using the `DatetimeOutputParser`
If you need to generate output in the form of a date or time, the `DatetimeOutputParser` from LangChain simplifies the process.

The **format of the `DatetimeOutputParser`** can be specified by referring to the table below.
| Format Code | Description           | Example              |
|--------------|-----------------------|----------------------|
| %Y           | 4-digit year          | 2024                 |
| %y           | 2-digit year          | 24                   |
| %m           | 2-digit month         | 07                   |
| %d           | 2-digit day           | 04                   |
| %H           | 24-hour format hour   | 14                   |
| %I           | 12-hour format hour   | 02                   |
| %p           | AM or PM              | PM                   |
| %M           | 2-digit minute        | 45                   |
| %S           | 2-digit second        | 08                   |
| %f           | Microsecond (6 digits)| 000123               |
| %z           | UTC offset            | +0900                |
| %Z           | Timezone name         | KST                  |
| %a           | Abbreviated weekday   | Thu                  |
| %A           | Full weekday name     | Thursday             |
| %b           | Abbreviated month     | Jul                  |
| %B           | Full month name       | July                 |
| %c           | Full date and time    | Thu Jul 4 14:45:08 2024 |
| %x           | Full date             | 07/04/24             |
| %X           | Full time             | 14:45:08             |

In [4]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate

# Initialize the output parser
output_parser = DatetimeOutputParser()

# Specify date format
date_format = "%Y-%m-%d"
output_parser.format = date_format

# Get format instructions
format_instructions = output_parser.get_format_instructions()

# Create answer template for user questions
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

# Create a prompt from the template
prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": format_instructions,
    },  # Use parser's format instructions
)

print(format_instructions)
print("-----------------------------------------------\n")
print(prompt)

Write a datetime string that matches the following pattern: '%Y-%m-%d'.

Examples: 381-04-29, 649-02-27, 2018-03-28

Return ONLY this string, no other words!
-----------------------------------------------

input_variables=['question'] input_types={} partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 381-04-29, 649-02-27, 2018-03-28\n\nReturn ONLY this string, no other words!"} template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:'


In [5]:
# Combine the prompt, chat model, and output parser into a chain
chain = prompt | llm | output_parser

# Call the chain to get an answer to the question
output = chain.invoke({"question": "The year Google was founded"})

print(output)
print(type(output))

1998-09-04 00:00:00
<class 'datetime.datetime'>


In [6]:
# Convert the result to a string
output.strftime(date_format)

'1998-09-04'

## Using `DatetimeOutputParser` in `astream`
Refer to the [user-defined generator](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/13-LangChain-Expression-Language/09-Generator.ipynb) to create a generator function.

Let's create a simple example that converts `astream` output to `datetime` objects using a generator function.



In [7]:
from langchain_core.output_parsers.string import StrOutputParser
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai.chat_models.base import ChatOpenAI
import datetime
from typing import AsyncIterator, List

# Initialize the output parser
output_parser = DatetimeOutputParser()

# Specify date format
date_format = "%Y-%m-%d"
output_parser.format = date_format

# Get format instructions
format_instructions = output_parser.get_format_instructions()

# Create answer template for user questions
template = (
    "Answer the users question:\n\n"
    "#Format Instructions: \n{format_instructions}\n"
    "Write a comma-separated list of 5 founding years of companies similar to: {company}"
)

# Create a prompt from the template
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": format_instructions},
)

# Initialize the ChatOpenAI model with temperature set to 0.0
model = ChatOpenAI(temperature=0.0, model_name="gpt-4o-mini")

# Create a chain combining the prompt, model, and string output parser
str_chain = prompt | model | StrOutputParser()


# Define an asynchronous function to convert strings to datetime objects
async def convert_strings_to_datetime(
    input: AsyncIterator[str],
) -> AsyncIterator[List[datetime.datetime]]:
    buffer = ""
    async for chunk in input:
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            date_str = buffer[:comma_index].strip()
            date_obj = output_parser.parse(date_str)  # Convert to datetime object
            yield [date_obj]
            buffer = buffer[comma_index + 1 :]
    date_str = buffer.strip()
    if date_str:
        date_obj = output_parser.parse(
            date_str
        )  # Convert remaining buffer to datetime object
        yield [date_obj]


# Connect the str_chain and convert_strings_to_datetime in a pipeline
alist_chain = str_chain | convert_strings_to_datetime

In [8]:
# Use async for loop to stream data.
async for chunk in alist_chain.astream({"company": "Google"}):
    # Print each chunk and flush the buffer.
    print(chunk, flush=True)

[datetime.datetime(1998, 9, 4, 0, 0)]
[datetime.datetime(1994, 4, 4, 0, 0)]
[datetime.datetime(2004, 2, 4, 0, 0)]
[datetime.datetime(2003, 2, 4, 0, 0)]
[datetime.datetime(2001, 3, 1, 0, 0)]
